In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf 
from sklearn.preprocessing import MinMaxScaler


2022-03-17 10:34:47.550410: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-17 10:34:47.550440: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:

rating = pd.read_csv('books.csv', sep=';', encoding="latin-1")
user = pd.read_csv('users.csv', sep=';',encoding="latin-1")
book = pd.read_csv('ratings.csv', sep=';', encoding="latin-1")



### to do

- store predicitions for all users (for [i] in df show pred)
- function for returning the pred

In [4]:
# df.copy() lets you create a copy of the df in order to not show warnings
# df.unique() or df["Age"].unique creates an array which can be itirated over
# in order to return a df and not a series use two brackets: df_data = df_age[["Age"]]

### merging data and filtering by rating

In [5]:
# merge user rating and book data
book_rating = pd.merge(rating, book, on='ISBN')

#remove unwanted columns
cols = ['Year-Of-Publication', 'Publisher', 'Book-Author', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
book_rating.drop(cols, axis=1, inplace=True)

In [6]:
book_rating.head()

,ISBN,Book-Title,User-ID,Book-Rating
0,0195153448,Classical Mythology,2,0
1,0002005018,Clara Callan,8,5
2,0002005018,Clara Callan,11400,0
3,0002005018,Clara Callan,11676,8
4,0002005018,Clara Callan,41385,0


In [7]:
# filtering books that have had => 25 ratings

rating_count = (book_rating.
     groupby(by = ['Book-Title'])['Book-Rating'].
     count().
     reset_index().
     rename(columns = {'Book-Rating': 'RatingCount_book'})
     [['Book-Title', 'RatingCount_book']]
    )

threshold = 25
rating_count = rating_count.query('RatingCount_book >= @threshold')

In [8]:
book_rating.head()

,ISBN,Book-Title,User-ID,Book-Rating
0,0195153448,Classical Mythology,2,0
1,0002005018,Clara Callan,8,5
2,0002005018,Clara Callan,11400,0
3,0002005018,Clara Callan,11676,8
4,0002005018,Clara Callan,41385,0


In [9]:
# filtering users that have given => 20 ratings

user_rating = pd.merge(rating_count, book_rating, left_on='Book-Title', right_on='Book-Title', how='left')

user_count = (user_rating.
     groupby(by = ['User-ID'])['Book-Rating'].
     count().
     reset_index().
     rename(columns = {'Book-Rating': 'RatingCount_user'})
     [['User-ID', 'RatingCount_user']]
    )
    
threshold = 20
user_count = user_count.query('RatingCount_user >= @threshold')

combined = user_rating.merge(user_count, left_on = 'User-ID', right_on = 'User-ID', how = 'inner')

print('Number of unique books: ', combined['Book-Title'].nunique())
print('Number of unique users: ', combined['User-ID'].nunique())

Number of unique books:  5850
Number of unique users:  3192


In [10]:
# normalise the rating feature

scaler = MinMaxScaler()
combined['Book-Rating'] = combined['Book-Rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(combined['Book-Rating'].values.reshape(-1,1)))
combined['Book-Rating'] = rating_scaled

### create user - book matrix

In [11]:

# creating user - book matrix 

combined = combined.drop_duplicates(['User-ID', 'Book-Title'])
user_book_matrix = combined.pivot(index='User-ID', columns='Book-Title', values='Book-Rating')
user_book_matrix.fillna(0, inplace=True)
users = user_book_matrix.index.tolist()
books = user_book_matrix.columns.tolist()
user_book_matrix = user_book_matrix.to_numpy()

In [12]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [13]:
# set up of network parameters, such as dimensions of each hidden layer 

num_input = combined['Book-Title'].nunique()
num_hidden_1 = 10
num_hidden_2 = 5

# initialise the TensorFlow placeholder
X = tf.placeholder(tf.float64, [None, num_input])

# weights and biases are randomly initialised
weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

### build the encode / decode model & create predictions

In [14]:
# building the encode & decode model 

def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2

In [15]:
# construct model and create predicitions

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
y_pred = decoder_op
y_true = X

In [16]:
# define loss function and optimiser, minimize the squared error, and define the evaluation metrics
loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)
eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
# initialising placeholders and variables
init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()

# df for storing predictions
pred_data = pd.DataFrame() 

### training the model

In [18]:
with tf.Session() as session:
    epochs = 10
    batch_size = 35

    session.run(init)
    session.run(local_init)

    num_batches = int(user_book_matrix.shape[0] / batch_size)
    user_book_matrix = np.array_split(user_book_matrix, num_batches)
    
    for i in range(epochs):

        avg_cost = 0
        for batch in user_book_matrix:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += l

        avg_cost /= num_batches

        print("epoch: {} Loss: {}".format(i + 1, avg_cost))

    user_book_matrix = np.concatenate(user_book_matrix, axis=0)

    preds = session.run(decoder_op, feed_dict={X: user_book_matrix})

    pred_data = pred_data.append(pd.DataFrame(preds))

    pred_data = pred_data.stack().reset_index(name='Book-Rating')
    pred_data.columns = ['User-ID', 'Book-Title', 'Book-Rating']
    pred_data['User-ID'] = pred_data['User-ID'].map(lambda value: users[value])
    pred_data['Book-Title'] = pred_data['Book-Title'].map(lambda value: books[value])
    
    keys = ['User-ID', 'Book-Title']
    index_1 = pred_data.set_index(keys).index
    index_2 = combined.set_index(keys).index

2022-03-17 10:34:55.809131: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-17 10:34:55.809170: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-17 10:34:55.809201: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (chiaseed): /proc/driver/nvidia/version does not exist
2022-03-17 10:34:55.810575: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


epoch: 1 Loss: 0.3345386674115946
epoch: 2 Loss: 0.2849593309910743
epoch: 3 Loss: 0.0722514365666679
epoch: 4 Loss: 0.003811933026024765
epoch: 5 Loss: 0.0031137868063524365
epoch: 6 Loss: 0.0029711890940114366
epoch: 7 Loss: 0.0029331980296154764
epoch: 8 Loss: 0.0029115860560233447
epoch: 9 Loss: 0.0028977240602086696
epoch: 10 Loss: 0.0028881112223613885


/tmp/ipykernel_85352/3131666910.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_data = pred_data.append(pd.DataFrame(preds))


### testing the model

In [19]:
# pred_data.shape

In [8]:
# top ten books by user
top_ten_ranked = pred_data[~index_1.isin(index_2)] 
top_ten_ranked = top_ten_ranked.sort_values(['User-ID', 'Book-Rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('User-ID').head(10)

NameError: name 'pred_data' is not defined

In [7]:
# top ten for chosen user
top_ten_ranked.loc[top_ten_ranked['User-ID'] == 278582]

NameError: name 'top_ten_ranked' is not defined

In [2]:
# df_pred = top_ten_ranked.loc[top_ten_ranked['User-ID'] == 278582]

In [3]:
# other books rated by chosen user

# book_rating.loc[book_rating['User-ID'] == 278582].sort_values(by=['Book-Rating'], ascending=False)

In [4]:
# pred_data.head(235)

## cleaning data

#### df books

In [9]:
# removing image columns
# df_books_cleaned = df_books[['ISBN','Book-Title', 'Book-Author','Year-Of-Publication','Publisher']]

In [ ]:
df_books_cleaned.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company


In [ ]:
# clean all rows that contain some nan
df_books_cleaned = df_books_cleaned.dropna(how='any')

In [ ]:
# books that are published between 1950 to 2016

df_books_cleaned = df_books_cleaned[(df_books_cleaned['Year-Of-Publication'] > 1950) & (df_books_cleaned['Year-Of-Publication'] <= 2016)]

In [ ]:
# regex-based replacement of certain characters
df_books_cleaned['Book-Author'] = df_books_cleaned['Book-Author'].str.replace(r'[^\w\s]+', '')

/tmp/ipykernel_7304/1979341940.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_books_cleaned['Book-Author'] = df_books_cleaned['Book-Author'].str.replace(r'[^\w\s]+', '')


In [ ]:
# df still contains shit like this: ('A MunnÃJordÃ\xa0', 1) --- not solved

x,y = np.unique(df_books_cleaned['Book-Author'], return_counts=True)

[(x[i],y[i]) for i in range(len(x))] 

[(' D Chiel', 1),
 (' J D Landis', 1),
 (' Mimma Balia', 1),
 ('142 moms from all over the world', 1),
 ('1977 Inaugural Committee US', 1),
 ('1st Graders of Samuel s Nixon School Carnegie Penn', 1),
 ('21st Century Publishing', 1),
 ('73 Magazine Editors', 1),
 ('A  C Black Ltd', 1),
 ('A  E Television Network', 2),
 ('A A  Winnie the Pooh Milne', 1),
 ('A A  WinnieThePooh Milne', 2),
 ('A A Attanasio', 9),
 ('A A Carr', 1),
 ('A A Gill', 1),
 ('A A Hoehing', 1),
 ('A A Hoehling', 2),
 ('A A Milne', 47),
 ('A A Sveshnikov', 1),
 ('A A Winnie the Pooh Milne', 1),
 ('A Adams', 1),
 ('A Alice', 1),
 ('A Alli', 1),
 ('A Alvarez', 3),
 ('A Anatoli', 1),
 ('A AndersonEvangelista', 1),
 ('A Avalon', 1),
 ('A B C Whipple', 1),
 ('A B Guthrie', 2),
 ('A B Jr Guthrie', 2),
 ('A B Simpson', 2),
 ('A B Yehoshua', 5),
 ('A Balfour', 1),
 ('A Barrie', 2),
 ('A Bartlett Giamatti', 1),
 ('A Bates', 7),
 ('A Blue Mountain Arts Collection', 1),
 ('A Breton', 1),
 ('A Brill', 1),
 ('A Bronte', 1),
 ('A 

#### df users

In [ ]:
# cleaned all rows that contained some nan
df_users_cleaned = df_users.dropna(how='any')

In [ ]:
# removing location column
df_users_cleaned = df_users_cleaned[['User-ID','Age']]

#### df ratings

In [ ]:
# eliminate nans first
df_ratings_cleaned = df_ratings.dropna(how='any')

In [ ]:
# undersampling the dataset by reducing 0-rating

# Put all the 0's in a separate dataset
df_without_zero = df_ratings_cleaned.loc[df_ratings_cleaned['Book-Rating'] != 0]

#Randomly select  observations from the non-fraud (majority class)
df_with_zero = df_ratings_cleaned.loc[df_ratings_cleaned['Book-Rating'] == 0].sample(n=45000,random_state=42)

# Concatenate both dataframes again
df_undersampled_ratings = pd.concat([df_without_zero, df_with_zero])
df_undersampled_ratings

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6
...,...,...,...
511086,124048,0425195473,0
964683,232964,0836220870,0
617446,149306,1930252501,0
214844,50205,0515114863,0


In [ ]:
# distribution of ratings
df_undersampled_ratings.groupby(['Book-Rating']).count() / len(df_undersampled_ratings) * 100 

In [ ]:
# count of ratings
df_undersampled_ratings.groupby(['Book-Rating']).count() 